## Topic model

In [3]:
import pandas as pd
import numpy as np

from test_model import (get_patent_fields_list, get_ml_patents, 
                        create_title_abstract_col,trim_data, 
                        structure_dataframe, partition_dataframe, 
                        build_pipeline, process_docs, pat_inv_map, get_topics)
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary, mmcorpus
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models.ldamodel import LdaModel
from gensim.models import AuthorTopicModel
from gensim.test.utils import common_dictionary, datapath, temporary_file
from smart_open import smart_open

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

import json
from pandas.io.json import json_normalize
import requests
import re
import os
import calendar
import requests
from bs4 import BeautifulSoup
import pickle

import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint

%load_ext autoreload
%autoreload 2

In [4]:
# TODO (Lee) - resolve deprecation warnings
import warnings; warnings.simplefilter('ignore')

In [5]:
np.random.seed(3)

### Acquire data

In [6]:
%%capture
# above '%%capture' supresses output of large output of json response

# query PatentsView API to acquire ML patent dataset as list of dictionaries 
raw_data_1000 = get_ml_patents(pats_per_page=1000)
raw_data_2000 = get_ml_patents(pats_per_page=2000)

In [7]:
# pickle raw_data_1000 and raw_data_2000
with open('/Users/lee/Documents/techniche/techniche/data/raw_data_1000', 'wb') as f:
    pickle.dump(raw_data_1000, f)   
with open('/Users/lee/Documents/techniche/techniche/data/raw_data_2000', 'wb') as f:
    pickle.dump(raw_data_2000, f)

# uncomment below to de-serialize pickled list of dictionaries
# with open('/Users/lee/Documents/techniche/techniche/data/raw_data_1000', 'rb') as f:
#     raw_data_1000 = pickle.load(f) 
# with open('/Users/lee/Documents/techniche/techniche/data/raw_data_2000', 'rb') as f:
#     raw_data_2000 = pickle.load(f)

# uncomment to view first dictionary in raw_data_1000
# raw_data_1000[0]

#### Acquire data - Structure data - 1000 patent documents

In [8]:
# define desired keys/columns as criteria to subset dataset
retained_keys = ['patent_number', 'patent_date', 'patent_title',
                 'patent_abstract', 'inventors']

In [9]:
# subset JSON dict dataset of full api response by desired keys/columns
data_1000 = trim_data(data=raw_data_1000, keys=retained_keys)

# TODO (Lee) review naming conv duplication - create item in dict by concatenating patent_title and patent_abstract
data_1000 = create_title_abstract_col(data=data_1000)

# convert dataframe from subsetted dict, organize columns and sort by patent_date
df_1000 = structure_dataframe(data=data_1000)

# partition df_1000 into train and test dataframes
data_train_1000, data_test_1000 = partition_dataframe(df_1000, .8)

# convert dataframes (full, train, test) to list format required by model
text_data_1000 = df_1000.patent_title_abstract.tolist()
text_train_1000 = data_train_1000.patent_title_abstract.tolist()
text_test_1000 = data_test_1000.patent_title_abstract.tolist()

# TODO (Lee) - convert text target in JSON response to list w/o dataframe step
text_list_1000 = []
for i in data_1000:
    text_list_1000.append(i['patent_title_abstract'])
# text_list

In [10]:
type(raw_data_1000)

list

#### Acquire data - Structure data - 2000 patent documents

In [11]:
# subset dataset of full api response by desired keys/columns
data_2000 = trim_data(data=raw_data_2000, keys=retained_keys)

# create dict item by concatenating patent_title and patent_abstract
data_2000 = create_title_abstract_col(data=data_2000)

# create dataframe, organize columns and sort by patent_date
df_2000 = structure_dataframe(data=data_2000)

# partition dataframe
data_train_2000, data_test_2000 = partition_dataframe(df_2000, .8)

# convert dataframe to list format required by model
text_data_2000 = df_2000.patent_title_abstract.tolist()
text_train_2000 = data_train_2000.patent_title_abstract.tolist()
text_test_2000 = data_test_2000.patent_title_abstract.tolist()

# TODO (Lee) - convert json response to list, w/o dataframe step
text_list_2000 = []
for i in data_2000:
    text_list_2000.append(i['patent_title_abstract'])
# text_list

### Pre-process text data

In [12]:
# uncomment to download standard stop words from Spacy
# !python -m spacy download en

# update path with location to save stopwords
path_stopwords = '/Users/lee/Documents/techniche/techniche/data/stopwords/english'
stop_words = stopwords.words(path_stopwords)

# create text pre-processing pipeline to tokenize, clean and lower text
nlp = build_pipeline()
print(nlp.pipe_names)

# TODO (Lee) # pre-process documents via json-to-list workflow above
# processed_docs_1 = process_docs(text_list)

# pre-process documents via json-to-df-to-list workflow above
processed_docs_1000train = process_docs(text_train_1000)

### Build corpus and dictionary

# build dictionary
id_to_word_1000train = Dictionary(processed_docs_1000train)

# save/pickle dictionary for subsequent use
# update path with location to save pickled model
path_pickle_id_to_word = '/Users/lee/Documents/techniche/techniche/data/id_to_word_1000train.pkl'
pickle.dump(id_to_word_1000train, open(path_pickle_id_to_word,'wb'))

# apply term-doc freq (list of (token_id, token_count) tuples) to docs
corpus_1000train = [id_to_word_1000train.doc2bow(doc) for doc in processed_docs_1000train]

# uncomment below to create/view formatted corpus
# formatted_corpus_1000 = [[(id_to_word[id], freq) for id, freq in text] for text in corpus_1000train]
# formatted_corpus_1000
# id_to_word_1000train.token2id

['tagger', 'parser', 'ner']


In [13]:
id_to_word_1000train

## Train model #1: Genism LDA model
Model #1: implementation: Gensim LDAmodel; k_topics=5; n_docs=1000, partition = 80/20

In [14]:
# construct model #1
# TODO (Lee) - resolve deprecation warnings
model_1 = LdaModel(corpus=corpus_1000train,
                   id2word=id_to_word_1000train,
                   num_topics=5, 
                   random_state=100,
                   update_every=1,
                   chunksize=100,
                   passes=10,
                   alpha='auto',
                   per_word_topics=True)

### Model #1 - Explore and visualize

In [15]:
# explore topics visually
pyLDAvis.enable_notebook()
viz_topics_model_1 = pyLDAvis.gensim.prepare(model_1, corpus_1000train, id_to_word_1000train)
# viz_topics_model_1

In [16]:
# uncomment below to view keywords in n topics in corpus
# pprint(model_1.print_topics())

In [17]:
# uncomment below to view most important keywords and weight of given topic w/ idx 0
# pprint(model_1.print_topic(4))

### Model #1 - Evaluate
We evaluate models using coherence and perplexity metrics. As unsupervised learning task, no labels with which to evaluate the "expected" prediction. There is an open research agenda on LDA evaluation approaches (intrinsic vs extrinsic; machine vs human-interpretable, etc., task-specific). 

#### Model #1 - Evaluate - Pre-process test set

In [18]:
# TODO (Lee) pre-process 1000 patents from df-to-list worfklow above
processed_docs_1000test = process_docs(text_test_1000)

# build dictionary with dataset of 1000 patents
id_to_word_1000test = Dictionary(processed_docs_1000test)

# apply term-doc frequency (list of (token_id, token_count) tuples) to 1000 patents
corpus_1000test = [id_to_word_1000test.doc2bow(doc) for doc in processed_docs_1000test]

#### Model #1 - Evaluate - Coherence
Calculate topic coherence for topic models with 4-step coherence pipeline (segmentation, probability estimation, confirmation measure, aggregation) from Roeder et al., 2015. "Exploring the space of topic coherence measures", WSDM '15 Proceedings of the Eighth ACM International Conference on Web Search and Data Mining (WSDM) 2015, 399-408.

In [19]:
# calculate coherence metric for train set ((n = 800 docs/1000 docs total in dataset))
coherence_model_1train = CoherenceModel(model=model_1, 
                                        texts=processed_docs_1000train,
                                        dictionary=id_to_word_1000train,
                                        coherence='c_v')
coherence_model_1train_get = coherence_model_1train.get_coherence()
print(coherence_model_1train_get)

0.3557055926610265


In [20]:
# calculate coherence metric for test_set (n = 200 docs/1000 docs total in dataset)
coherence_model_1test = CoherenceModel(model=model_1, 
                                       texts=processed_docs_1000test, 
                                       dictionary=id_to_word_1000test, 
                                       coherence='c_v')
coherence_model_1test_get = coherence_model_1test.get_coherence()
print(coherence_model_1test_get)

0.6020573135746659


In [21]:
# calculate coherence metric for each of the n topics in the test set
coherence_model_1_per_topic = coherence_model_1test.get_coherence_per_topic()

# uncomment below to print coherence_model_1_per_topic
# print(coherence_model_1_per_topic)

#### Model #1 - Evaluate - Perplexity
Calculate perplexity metric. Metric calculates and returns per-word likelihood bound using a chunk of documents as evaluation corpus. Output calculated statistics, including the perplexity=2^(-bound), to log at INFO level. Returns the variational bound score calculated for each word

In [22]:
# calculate perplexity metric for model_1 train set (1000 patents)
perplexity_model_1train = model_1.log_perplexity(corpus_1000train)
print(perplexity_model_1train)

-7.091984309948656


In [23]:
# calculate perplexity metric for model_1 test set (1000 patents)
perplexity_model_1test = model_1.log_perplexity(corpus_1000test)
print(perplexity_model_1test)

-9.661613601356644


### Model #1 - Predict

#### Model #1 - Predict - Pickle model

In [24]:
# save/pickle model #1 for subsequent use
# update path with location to save pickled model
path_pickle_model_1 = '/Users/lee/Documents/techniche/techniche/data/model_lda_1.pkl'
pickle.dump(model_1, open(path_pickle_model_1,'wb'))

In [25]:
# uncommment below to load pickled model #1
# model_1 = pickle.load(open(path_pickle_model_1

#### Model #1 - inference

In [26]:
## Test model #1 on 2 new text w/ two strings

# define example text_input #1, expressing keyword-type search
text_input_1 = 'smart assistant transformer model translation'.split()

# define example text_input #1, expressing technical details in job post
text_input_2 = """At the Siri International team within Apple we bring the
Siri intelligent assistant to our customers worldwide in over 40 languages
and dialects. Join us, and tackle some of the most challenging problems in
natural language processing and large scale applied machine learning. You 
will build cutting edge natural language understanding technologies and 
deploy them on a global scale. Your work will advance and shape the future
vision of our multi-lingual, multi-cultural Siri assistant, and Search 
applications used by millions across the world Key Qualifications Extensive
track record of scientific research in NLP and Machine Learning, or similar
experience in developing language technologies for shipping products.
Strong coding and software engineering skills in a mainstream programming 
language, such as Python, Java, C/C++. Familiarity with NLP/ML tools and 
packages like Caffe, pyTorch, TensorFlow, Weka, scikit-learn, nltk, etc.
Practical experience building production quality applications related to 
natural language processing and machine learning. In-depth knowledge of 
machine learning algorithms and ability to apply them in data driven natural
language processing systems. Ability to quickly prototype ideas / solutions,
perform critical analysis, and use creative approaches for solving complex 
problems. Attention to detail and excellent communication skills. Description
We are looking for a highly motivated technologist with a strong background 
in Natural Language Processing and Machine Learning research. The ideal 
candidate will have a strong track record of taking research ideas to 
real-world applications. In this position you will apply your problem solving
skills to challenges and opportunities within Siri International, which 
involves development of large-scale language technologies for many natural
languages worldwide. The primary responsibility of this role is to conduct
research and develop innovative machine learning, artificial intelligence 
and NLP solutions for multi-lingual conversational agents. You will have 
the opportunity to investigate cutting edge research methods that will 
improve customer experience of our products and enable our engineers to 
scale these technologies across a variety of natural languages. You will 
also provide technical leadership and experiment-driven insights for 
engineering teams on their machine learning modeling and data decisions. 
You will play a central role in defining the future technical directions 
of Siri International through quick prototyping, critical analysis and 
development of core multi-lingual NLP technologies. Education & Experience
PhD in Machine Learning, Statistics, Computer Science, Mathematics or 
related field with specialization in natural language processing and/or 
machine learning, OR * Masters degree in a related field with a strong 
academic/industrial track record. * Hands-on research experience in an 
academic or industrial setting.""".split()

In [27]:
%%capture
# pass text through pre-process pipeline
id_to_word_1000train.doc2bow(text_input_1)
id_to_word_1000train.doc2bow(text_input_2)

In [28]:
predict_input_1 = get_topics(id_to_word_1000train.doc2bow(text_input_1), model_1, k=10)
# uncomment below to view predict_input_1
# predict_input_1

In [29]:
predict_input_2 = get_topics(id_to_word_1000train.doc2bow(text_input_2), model_1, k=10)
# uncomment below to view predict_input_2
# predict_input_2

## Train model #2: Genism LDAMallet model
Model #2: implementation: Gensim LDAMallet wrapper around LDA Mallet model; 
          k_topics=5; 
          n_docs=1000; 
          partition = 80/20

In [30]:
# uncomment to download Mallet topic model
# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# update path
path_mallet = '/Users/lee/Documents/techniche/techniche/data/mallet-2.0.8/bin/mallet'

In [31]:
# construct model #2
# TODO (Lee) - resolve deprecation warnings
model_2 = gensim.models.wrappers.LdaMallet(path_mallet, 
                                           corpus=corpus_1000train, 
                                           num_topics=5, 
                                           id2word=id_to_word_1000train)

### Model #2 - Explore and visualize

In [32]:
# TODO (Lee) - visualization with LDA Mallet
# explore topics visually
# pyLDAvis.enable_notebook()
# viz_topics_model_2 = pyLDAvis.prepare(model_2, corpus_1000train, id_to_word_1000train)
# viz_topics_model_2

In [33]:
# uncomment below to view keywords in n topics in corpus
# pprint(model_2.print_topics())

In [34]:
# uncomment below to view most important keywords and weight for given topic (idx 0)
# uncomment below to view
# pprint(model_2.print_topic(4))

### Model #2- Evaluate

#### Model #2 - Evaluate - Coherence

In [35]:
# calculate coherence metric for train set ((n = 800 docs/1000 docs total in dataset))
coherence_model_2train = CoherenceModel(model=model_2, 
                                        texts=processed_docs_1000train,
                                        dictionary=id_to_word_1000train,
                                        coherence='c_v')
coherence_model_2train_get = coherence_model_2train.get_coherence()
print(coherence_model_2train_get)

0.37918464892359377


In [36]:
# calculate coherence metric for test_set (n = 200 docs/1000 docs total in dataset)
coherence_model_2test = CoherenceModel(model=model_2, 
                                       texts=processed_docs_1000test, 
                                       dictionary=id_to_word_1000test, 
                                       coherence='c_v')
coherence_model_2test_get = coherence_model_2test.get_coherence()
print(coherence_model_2test_get)

0.6205180192789347


In [37]:
# calculate coherence metric for each of the n topics in the test set
coherence_model_2_per_topic = coherence_model_2test.get_coherence_per_topic()
# print(coherence_model_2_per_topic)

## Train Model #3: Gensim LDA model
Model #1: implementation: Gensim LDAmodel; k_topics=10; n_docs=1000, partition = 80/20
This model increases the k_topics from 5 to 10, relative to model #1 above

In [38]:
# construct model #3
# TODO (Lee) - resolve deprecation warnings
model_3 = LdaModel(corpus=corpus_1000train,
                   id2word=id_to_word_1000train,
                   num_topics=10, 
                   random_state=100,
                   update_every=1,
                   chunksize=100,
                   passes=10,
                   alpha='auto',
                   per_word_topics=True)

### Model #3 - Explore and visualize

In [39]:
# explore topics visually
pyLDAvis.enable_notebook()
viz_topics_model_3 = pyLDAvis.gensim.prepare(model_3, corpus_1000train, id_to_word_1000train)
# viz_topics_model_3

In [40]:
# keywords in n topics in corpus
# uncomment below to view
# pprint(model_3.print_topics())

In [41]:
# most important keywords, and the respective weight, that form topic with index 0
# uncomment below to view
# pprint(model_3.print_topic(4))

### Model #3 - Evaluate

#### Model #3 - Evaluate - Coherence

In [42]:
# calculate coherence metric for train set ((n = 800 docs/1000 docs total in dataset))
coherence_model_3train = CoherenceModel(model=model_3, 
                                        texts=processed_docs_1000train,
                                        dictionary=id_to_word_1000train,
                                        coherence='c_v')
coherence_model_3train_get = coherence_model_3train.get_coherence()
print(coherence_model_3train_get)

0.3628245805478861


In [43]:
# calculate coherence metric for test_set (n = 200 docs/1000 docs total in dataset)
coherence_model_3test = CoherenceModel(model=model_3, 
                                       texts=processed_docs_1000test, 
                                       dictionary=id_to_word_1000test, 
                                       coherence='c_v')
coherence_model_3test_get = coherence_model_3test.get_coherence()
print(coherence_model_3test_get)

0.6081902191345155


In [44]:
# calculate coherence metric for each of the n topics in the test set
coherence_model_3_per_topic = coherence_model_3test.get_coherence_per_topic()
# print(coherence_model_1_per_topic)

#### Model #3 - Evaluate - Perplexity

In [45]:
# calculate perplexity metric for model_1 train set
perplexity_model_3train = model_3.log_perplexity(corpus_1000train)
print(perplexity_model_3train)

-7.0676967060500635


In [46]:
# calculate perplexity metric for model_1 test set
perplexity_model_3test = model_3.log_perplexity(corpus_1000test)
print(perplexity_model_3test)

-9.945866263106378


### Model #3 - Predict

#### Model #3 - Predict - Pickle model

In [47]:
# pickle model #3
# # update path with location to save pickled model
path_pickle_model_3 = '/Users/lee/Documents/techniche/techniche/data/model_3.pkl'
pickle.dump(model_3, open(path_pickle_model_3,'wb'))

In [48]:
model_3 = pickle.load(open(path_pickle_model_3,'rb'))

#### Model #3 - inference

In [49]:
predict_input_1_model_3 = get_topics(id_to_word_1000train.doc2bow(text_input_1), model_3, k=10)
# uncomment below to view predict_input_1_model_3
# predict_input_1_model_3

In [50]:
predict_input_2_model_3 = get_topics(id_to_word_1000train.doc2bow(text_input_2), model_3, k=10)
# uncomment below to view predict_input_2_model_3
# predict_input_2_model_3

## Train Model #4: Gensim LDA model
Model #4: implementation: Gensim LDAmodel; k_topics=15; n_docs=1000, partition = 80/20
This model increases the k_topics to 15, relative to model #1 and model #3 above

In [51]:
# construct model #4
# TODO (Lee) - resolve deprecation warnings
model_4 = LdaModel(corpus=corpus_1000train,
                   id2word=id_to_word_1000train,
                   num_topics=15, 
                   random_state=100,
                   update_every=1,
                   chunksize=100,
                   passes=10,
                   alpha='auto',
                   per_word_topics=True)

### Model #4 - Explore and visualize

In [52]:
# explore topics visually
pyLDAvis.enable_notebook()
viz_topics_model_4 = pyLDAvis.gensim.prepare(model_4, corpus_1000train, id_to_word_1000train)
# viz_topics_model_1

In [53]:
# uncomment below to view keywords in n topics in corpus
# pprint(model_4.print_topics())

In [54]:
# uncomment below to view most important keywords and weight for given topic (idx 0)
# pprint(model_4.print_topic(4))

### Model #4 - Evaluate

#### Model #4 - Evaluate - Coherence

In [55]:
# calculate coherence metric for train set ((n = 800 docs/1000 docs total in dataset))
coherence_model_4train = CoherenceModel(model=model_4,
                                        texts=processed_docs_1000train,
                                        dictionary=id_to_word_1000train,
                                        coherence='c_v')
coherence_model_4train_get = coherence_model_4train.get_coherence()
print(coherence_model_4train_get)

0.37342906623443217


In [56]:
#TODO (Lee)
# calculate coherence metric for test_set (n = 200 docs/1000 docs total in dataset)
# coherence_model_4test = CoherenceModel(model=model_4,
#                                        texts=processed_docs_1000test,
#                                        dictionary=id_to_word_1000test,
#                                        coherence='c_v')
# coherence_model_4test_get = coherence_model_4test.get_coherence()
# print(coherence_model_4test_get)

In [57]:
#TODO (Lee)
# calculate coherence metric for each of the n topics in the test set
# coherence_model_4_per_topic = coherence_model_4test.get_coherence_per_topic()
# print(coherence_model_1_per_topic)

#### Model #4 - Evaluate - Perplexity

In [58]:
# calculate perplexity metric for model_1 train set
perplexity_model_3train = model_3.log_perplexity(corpus_1000train)
print(perplexity_model_3train)

-7.067692260355805


In [59]:
# calculate perplexity metric for model_1 test set
perplexity_model_3test = model_3.log_perplexity(corpus_1000test)
print(perplexity_model_3test)

-9.945866293005516


### Model #4 - Predict

#### Model #4 - Predict - Pickle model

In [60]:
# pickle model
# # update path with location to save pickled model
path_pickle_model_4 = '/Users/lee/Documents/techniche/techniche/data/model_4.pkl'
pickle.dump(model_4, open(path_pickle_model_4,'wb'))

In [61]:
model_4 = pickle.load(open(path_pickle_model_4,'rb'))

#### Model #4 - inference

In [62]:
predict_input_1_model_4 = get_topics(id_to_word_1000train.doc2bow(text_input_1), model_4, k=10)
# uncomment below to view predict_input_1_model_4
# predict_input_1_model_4

In [63]:
predict_input_2_model_4 = get_topics(id_to_word_1000train.doc2bow(text_input_2), model_4, k=10)
# uncomment below to view predict_input_2_model_4
# predict_input_2_model_4

## Train Model #5: Author-topic model
Model #4: implementation: Gensim AuthorTopicModel; k_topics=15; n_docs=1000, partition = 80/20
This model increases the k_topics to 15, relative to model #1 and model #3 above

In [64]:
# uncomment to view quick visual index to patent number mapping
# for i in raw_data_1000:
#     print(raw_data_1000.index(i), i['patent_number'])

In [65]:
# TODO (Lee) review pat_inv_map workflow
# partitions data_1000 to size of training set (80/20 split so grabs first 800 rows)
data_800 = data_1000[:800]

# create inventor-to-doc mapping from original list of dicts in json api response
pat2inv = pat_inv_map(data_800)

#### Construct author-topic model

In [66]:
# construct author-topic model
model_at = AuthorTopicModel(corpus=corpus_1000train,
                            doc2author=pat2inv,
                            id2word=id_to_word_1000train)

In [67]:
# construct vectors for authors
author_vecs = [model_at.get_author_topics(author) for author in model_at.id2author.values()]
author_vecs

[[(46, 0.9643839432941929)],
 [(65, 0.9639355662340786)],
 [(57, 0.9155596112480439)],
 [(48, 0.9106047601892854)],
 [(84, 0.9136289757853294)],
 [(25, 0.913399447892555)],
 [],
 [(55, 0.9223575854488589)],
 [(15, 0.9237006044025423)],
 [(23, 0.9232493512012585)],
 [(4, 0.913219716362475)],
 [(17, 0.46472217075139427), (21, 0.4447310778409392)],
 [(15, 0.31823650809958015), (73, 0.5926160784000312)],
 [(68, 0.9095887391735005)],
 [(99, 0.9122984850435393)],
 [(33, 0.9109323195414232)],
 [(82, 0.8669217446428301)],
 [(92, 0.8696849365356564)],
 [(64, 0.8710193865525495)],
 [(45, 0.8637760143693869)],
 [(43, 0.8714637378257775)],
 [(20, 0.8657615275181453)],
 [(26, 0.8700237345562354)],
 [(60, 0.8571555715609285)],
 [(8, 0.30676073627275546), (39, 0.5288994175189405)],
 [(66, 0.8507292367745055)],
 [(34, 0.8522793561659548)],
 [(50, 0.8555332190378276)],
 [(62, 0.8543287019381355)],
 [(0, 0.8539629604031524)],
 [(18, 0.8558094991278395)],
 [(81, 0.8502098634803396)],
 [(51, 0.82966107001

In [68]:
# retrieve topic distribution for author using use model[name] syntax
# each topic has a probability of being expressed given the particular author, 
# but only the ones above a certain threshold are displayed

model_at['7788103-1']

[(3, 0.8530755969508286), (86, 0.13368115980592937)]

In [69]:
# def show_author(name):
#     print('\n%s' % name)
#     print('Docs:', model.author2doc[name])
#     print('Topics:')
#     pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])

In [70]:
# build mapping from inventor to patent
inv2pat = gensim.models.atmodel.construct_author2doc(pat2inv)

### Model #X - Predict

In [71]:
# prediction functions that take input of new text string, and predict topic distribution

## Train Model #6: Gensim LDA model
Model #6: implementation: Gensim LDAmodel; k_topics=5; n_docs=1000, partition = 80/20
This model decreases the k_topics to 5 for human interpretability in webapp

In [72]:
# construct model #6
# TODO (Lee) - resolve deprecation warnings
model_6 = LdaModel(corpus=corpus_1000train,
                   id2word=id_to_word_1000train,
                   num_topics=5, 
                   random_state=100,
                   update_every=1,
                   chunksize=100,
                   passes=10,
                   alpha='auto',
                   per_word_topics=True)

#### Model #6 - Evaluate - Coherence

In [73]:
# calculate coherence metric for train set ((n = 800 docs/1000 docs total in dataset))
coherence_model_6train = CoherenceModel(model=model_6,
                                        texts=processed_docs_1000train,
                                        dictionary=id_to_word_1000train,
                                        coherence='c_v')
coherence_model_6train_get = coherence_model_6train.get_coherence()
print(coherence_model_6train_get)

0.3557055926610265


#### Model #6 - Evaluate - Perplexity

In [74]:
# calculate perplexity metric for model_6 train set
perplexity_model_6train = model_6.log_perplexity(corpus_1000train)
print(perplexity_model_6train)

-7.091984285741993


In [75]:
# calculate perplexity metric for model_6 test set
perplexity_model_6test = model_6.log_perplexity(corpus_1000test)
print(perplexity_model_6test)

-9.661613613602723


#### Model #6 - Predict - Pickle model

In [76]:
# pickle model
# # update path with location to save pickled model
path_pickle_model_6 = '/Users/lee/Documents/techniche/techniche/data/model_6.pkl'
pickle.dump(model_6, open(path_pickle_model_6,'wb'))

In [77]:
model_6 = pickle.load(open(path_pickle_model_6,'rb'))

#### Model #6 - inference

In [80]:
predict_input_1_model_6 = get_topics(id_to_word_1000train.doc2bow(text_input_1), model_6, k=3)
# uncomment below to view predict_input_1_model_6
predict_input_1_model_6

[(0,
  '0.027*"language" + 0.026*"one" + 0.025*"natural" + 0.024*"user" + 0.020*"first" + 0.020*"may" + 0.017*"data" + 0.015*"based" + 0.015*"processing" + 0.014*"system"'),
 (4,
  '0.025*"machine" + 0.024*"learning" + 0.023*"data" + 0.020*"image" + 0.019*"system" + 0.014*"network" + 0.012*"based" + 0.011*"may" + 0.011*"using" + 0.011*"security"'),
 (1,
  '0.029*"learning" + 0.029*"machine" + 0.028*"data" + 0.011*"system" + 0.011*"training" + 0.009*"based" + 0.008*"model" + 0.008*"apparatus" + 0.008*"methods" + 0.007*"systems"')]

In [79]:
predict_input_2_model_6 = get_topics(id_to_word_1000train.doc2bow(text_input_2), model_6, k=10)
# uncomment below to view predict_input_2_model_6
# predict_input_2_model_6